In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
#from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations
import idaes
from itertools import permutations, product

from fim_doe import *

import pyomo.common.unittest as unittest

In [2]:
import cvxpy as cp

from reactor_kinetics import *

(CVXPY) Aug 28 10:04:00 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')
(CVXPY) Aug 28 10:04:00 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')


## FIM reader

In [3]:
def dataframe_return(data):
    fim1 = []
    fim1.append(data['P1'].to_list())
    fim1.append(data['P2'].to_list())
    fim1.append(data['P3'].to_list())
    fim1.append(data['P4'].to_list())
    
    return fim1 

In [4]:
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

#t_measure = [0.125]



def FIM_reader(rank='onebyone'):
    
    FIM_set = []
    
    if rank=='onebyone':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
    elif rank=='onealong':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
        for idx, i in enumerate(t_measure):
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
        for idx, i in enumerate(t_measure):
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
            
    return FIM_set
        
FIM_set = FIM_reader(rank='onealong')
print(np.shape(FIM_set))
print(FIM_set[0][1][1])

(27, 4, 4)
0.0


In [5]:
TotalF = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        TotalF[i,j] = sum(FIM_set[k][i][j] for k in range(27))
        
print(TotalF)

print(np.trace(TotalF))
print(np.linalg.det(TotalF))
print(np.linalg.eigvals(TotalF))

[[  17.66733763   18.74907479  -30.09833141  -76.66335672]
 [  18.74907479   30.78667315  -30.04626402 -124.88329795]
 [ -30.09833141  -30.04626402   52.08798793  123.38481043]
 [ -76.66335672 -124.88329795  123.38481043  514.04755797]]
614.5895566797835
663.4981277559746
[5.87563028e+02 2.64907667e+01 9.71982177e-02 4.38563426e-01]


### Jacobian reader

In [6]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')

#print(jaco_info)


Q_stack = []

for i in range(27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    
    Q_stack.append(Q_res)
    
print(Q_stack)

[[0.0, 0.0, 0.0, 0.0], [-1.346695417017152, -1.665334536937735e-13, 2.223379846162832, -1.6653345369377348e-13], [-1.039297035228287, -2.220446049250313e-13, 2.004516103508741, -1.6653345369377348e-13], [-0.7733130998612002, -1.3877787807814457e-13, 1.63790005416059, -1.1102230246251565e-13], [-0.560819154400885, -1.249000902703301e-13, 1.2648388615049688, -6.938893903907228e-14], [-0.3990444064009696, -4.8572257327350605e-14, 0.9415433202331, -2.081668171172168e-14], [-0.2797955547643657, -5.5511151231257833e-14, 0.6830391077798161, -2.081668171172168e-14], [-0.1939053002679436, -3.1225022567582534e-14, 0.486126099753268, -3.4694469519536138e-15], [-0.1331100105811684, -1.3877787807814455e-14, 0.3409191457896289, 1.040834085586084e-14], [0.0, 0.0, 0.0, 0.0], [0.0199157340269318, -1.3752037136890438, -0.0468172929494192, 4.379651748244728], [-0.2728277373608145, -1.38740313152641, 0.1675143115261734, 4.866408693837143], [-0.4952849992631014, -1.4015562993399122, 0.4972483613988609, 5.3

### FIM verificator

## Covariance without the initial measurement

In [7]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')

#print(jaco_info)


Q_stack = []

for i in range(1,9):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(10,18):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(19,27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
print(Q_stack)
    
print(np.shape(Q_stack))

[[-1.346695417017152, -1.665334536937735e-13, 2.223379846162832, -1.6653345369377348e-13], [-1.039297035228287, -2.220446049250313e-13, 2.004516103508741, -1.6653345369377348e-13], [-0.7733130998612002, -1.3877787807814457e-13, 1.63790005416059, -1.1102230246251565e-13], [-0.560819154400885, -1.249000902703301e-13, 1.2648388615049688, -6.938893903907228e-14], [-0.3990444064009696, -4.8572257327350605e-14, 0.9415433202331, -2.081668171172168e-14], [-0.2797955547643657, -5.5511151231257833e-14, 0.6830391077798161, -2.081668171172168e-14], [-0.1939053002679436, -3.1225022567582534e-14, 0.486126099753268, -3.4694469519536138e-15], [-0.1331100105811684, -1.3877787807814455e-14, 0.3409191457896289, 1.040834085586084e-14], [0.0199157340269318, -1.3752037136890438, -0.0468172929494192, 4.379651748244728], [-0.2728277373608145, -1.38740313152641, 0.1675143115261734, 4.866408693837143], [-0.4952849992631014, -1.4015562993399122, 0.4972483613988609, 5.32136224081181], [-0.6436372848857008, -1.409

In [8]:
# variance 

variance = [1, # var_CA^2
            4, # var_CB^2
            8  # var_CC^2
           ]
covariance = [ 0.1, # cov_CA_CB | ti, 
               0.1, # cov_CA_CC | ti, 
               0.5 # cov_CB_CC | ti
]
'''
variance = [1,1,1]
covariance=[0,0,0]
'''
# construct Sigma
sigma = np.zeros((24, 24))

for i in range(24):
    #for j in range(27):
        
        #if i==j:
    if i<8:
        sigma[i,i]=variance[0]
        sigma[i,i+8]=covariance[0]
        sigma[i,i+16]=covariance[1]
    elif 8<= i < 16:
        sigma[i,i]=variance[1]
        sigma[i,i-8]=covariance[0]
        sigma[i,i+8]=covariance[2]
    elif i>=16:
        sigma[i,i]=variance[2]
        sigma[i,i-16]=covariance[1]
        sigma[i,i-8]=covariance[2]
                
            
# check eigen values to make sure it's PSD 
print(np.linalg.eigvals(sigma))

[0.99566752 8.06331241 3.94102007 8.06331241 8.06331241 3.94102007
 3.94102007 0.99566752 0.99566752 0.99566752 0.99566752 8.06331241
 8.06331241 0.99566752 8.06331241 8.06331241 8.06331241 0.99566752
 0.99566752 3.94102007 3.94102007 3.94102007 3.94102007 3.94102007]


In [9]:
sigma_inv = np.linalg.inv(sigma)

In [10]:
Q_matrice = np.asarray(Q_stack)
sigma_inv_mat = np.asarray(sigma_inv)

print(np.shape(Q_matrice))
print(np.shape(sigma_inv_mat))

(24, 4)
(24, 24)


In [11]:
fim_collection = []

#print(fim_collection)

for i in range(24):
    for j in range(24):
        unit = sigma_inv_mat[i,j]*np.matrix(Q_matrice[i,:]).T@np.matrix(Q_matrice[j,:])
        
        unit_list = [[0]*4 for i in range(4)] 
    
        for k in range(4):
            for l in range(4):
                unit_list[k][l] = unit[k,l]
        
        fim_collection.append(unit_list)
        
print(type(fim_collection[0]))
print(fim_collection[0])

print(np.shape(fim_collection))

<class 'list'>
[[1.8198936897068985, 2.2504953805640995e-13, -3.0046251741285146, 2.2504953805640992e-13], [2.2504953805640995e-13, 2.7829809436594326e-26, -3.7155439974034176e-13, 2.782980943659432e-26], [-3.004625174128514, -3.7155439974034176e-13, 4.9606042858804384, -3.715543997403417e-13], [2.2504953805640992e-13, 2.782980943659432e-26, -3.715543997403417e-13, 2.7829809436594315e-26]]
(576, 4, 4)


In [ ]:
print(fim_collection[575])

In [ ]:


fim = [[0,0,0,0] for i in range(4)]
for i in range(24):
    for j in range(24):
        fim_unit = 1*fim_collection[i*24+j]
        for k in range(4):
            for l in range(4):
                fim[k][l] += fim_unit[k][l]

print(fim)

In [ ]:
fim = Q_matrice.T@sigma_inv_mat@Q_matrice
print(fim)

## With a more generalized error covariance

In [12]:
no_measure = 3
no_t = 8

def eval_fim_1(y):
    #fim = cp.sum(Q_matrice[i].T@np.matrix([y[i,j]])@np.matrix([sigma_inv_mat[i,j]])@Q_matrice[j] for i in range(24) for j in range(24))
    #fim = cp.sum(y[i,j]*sigma_inv_mat[i,j]*(Q_matrice[i].T@Q_matrice[j]) for i in range(24))
    fim = Q_matrice.T@y@sigma_inv_mat@Q_matrice
    return fim 

def eval_fim_2(y):
    fim = [[0,0,0,0] for i in range(4)]
    
    for i in range(24):
        for j in range(24):
            Q_m1 = Q_matrice[i]
            Q_m2 = Q_matrice[j]
            #measure_matrix = np.matrix([y[i,j]])
            #sigma = np.matrix([sigma_inv_unit[i,j]])
            
            #fim_unit = Q_m1.T@measure_matrix@sigma@Q_m2
            #fim_unit = y[i,j]*sigma_inv_unit[i,j]*(Q_m1.T@Q_m2)
            fim_unit = y[i,j]*sigma_inv_unit[i,j]*np.dot(Q_m1[:, None], Q_m2[None, :])
        
            fim += fim_unit
    
    return fim 

def eval_fim(y):
    #fim = [[0,0,0,0] for _ in range(4)]
    #for i in range(24):
    #    for j in range(24):
    #        fim_unit = y[i,j]*fim_collection[i*24+j]
            #for k in range(4):
            #    for l in range(4):
            #fim[k][l] += fim_unit[k][l]
            #fim += fim_unit
    fim = sum(y[i,j]*fim_collection[i*24+j] for i in range(24) for j in range(24))
            
    return fim

def a_opt(y):
    fim = eval_fim(y)
    return cp.trace(fim)
    
def d_opt(y):
    fim = eval_fim(y)
    return cp.log_det(fim)

def e_opt(y):
    fim = eval_fim(y)
    return -cp.lambda_min(fim)

In [13]:
cost = [10]*8 + [6]*16

y_matrice = cp.Variable((24,24), nonneg=True)

p_cons = [sum(y_matrice[i,i]*cost[i] for i in range(24)) <= 100]

for j in range(24):
    p_cons += [y_matrice[j,j]<=1]
    #p_cons += [y_matrice[j,j]>=0]

for k in range(24):
    for l in range(24):
        p_cons += [y_matrice[k,l] <= y_matrice[k,k]]
        p_cons += [y_matrice[k,l] <= y_matrice[l,l]]
        p_cons += [y_matrice[k,k] + y_matrice[l,l] -1 <= y_matrice[k,l]]
        p_cons += [y_matrice.T == y_matrice]


obj = cp.Maximize(d_opt(y_matrice))

problem = cp.Problem(obj, p_cons)

In [14]:
print('First time: Optimal value:', problem.solve(verbose=True))

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Aug 28 10:04:54 PM: Your problem has 576 variables, 2329 constraints, and 0 parameters.
(CVXPY) Aug 28 10:04:55 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 28 10:04:55 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 28 10:04:55 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 28 10:04:55 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Aug 28 10:04:55 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr ->

(CVXPY) Aug 28 10:05:04 PM: 24  7.3e-10  3.0e-06  6.1e-14  1.00e+00   -1.632979820e+00  -1.632979818e+00  5.2e-10  2.14  
(CVXPY) Aug 28 10:05:04 PM: Optimizer terminated. Time: 2.24    
(CVXPY) Aug 28 10:05:04 PM: 
(CVXPY) Aug 28 10:05:04 PM: 
(CVXPY) Aug 28 10:05:04 PM: Interior-point solution summary
(CVXPY) Aug 28 10:05:04 PM:   Problem status  : PRIMAL_AND_DUAL_FEASIBLE
(CVXPY) Aug 28 10:05:04 PM:   Solution status : OPTIMAL
(CVXPY) Aug 28 10:05:04 PM:   Primal.  obj: -1.6329798201e+00   nrm: 2e+01    Viol.  con: 4e-09    var: 5e-09    barvar: 0e+00    cones: 5e-11  
(CVXPY) Aug 28 10:05:04 PM:   Dual.    obj: -1.6329798175e+00   nrm: 7e+01    Viol.  con: 0e+00    var: 1e-09    barvar: 3e-10    cones: 3e-17  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Aug 28 10:05:05 PM: Problem 

In [15]:
print(y_matrice.value)
#print(y[1].value)
#print(y[2].value)

[[9.99999978e-01 9.99999787e-01 9.99999883e-01 9.99999838e-01
  3.64945954e-07 8.40093018e-08 5.29666001e-08 4.22955533e-08
  9.99999953e-01 9.99999847e-01 9.99999643e-01 8.89343329e-01
  1.96434845e-06 9.99999671e-01 9.99999856e-01 9.99999904e-01
  9.99999889e-01 5.87555783e-01 1.59709523e-07 1.29022480e-07
  1.52991397e-07 2.62707781e-07 5.23099416e-01 9.99999687e-01]
 [9.99999787e-01 9.99999799e-01 9.99999737e-01 9.99999705e-01
  2.92121628e-07 5.40490409e-08 3.43069839e-08 2.78434118e-08
  9.99999780e-01 9.99999729e-01 9.99999535e-01 8.89343239e-01
  1.87712499e-06 9.99999561e-01 9.99999718e-01 9.99999751e-01
  9.99999741e-01 5.87555736e-01 1.09177829e-07 8.56074455e-08
  1.03873707e-07 1.97189237e-07 5.23099326e-01 9.99999575e-01]
 [9.99999883e-01 9.99999737e-01 9.99999894e-01 9.99999784e-01
  3.26905951e-07 6.10236933e-08 3.72904289e-08 2.98082504e-08
  9.99999874e-01 9.99999793e-01 9.99999619e-01 8.89343287e-01
  1.92285724e-06 9.99999624e-01 9.99999800e-01 9.99999841e-01
  9.99

### test solution 

In [16]:
sol = np.zeros((24,24))

for i in range(24):
    for j in range(24):
        sol[i,j] = y_matrice[i,j].value
        
        if sol[i,j] >0.99:
            sol[i,j] = 1
            
        if sol[i,j] <0.01:
            sol[i,j] = 0
            
        
        
print(list(sol[i,i] for i in range(24)))

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.8893433395380824, 0.0, 1.0, 1.0, 1.0, 1.0, 0.5875557934505837, 0.0, 0.0, 0.0, 0.0, 0.5230994268276815, 1.0]


In [17]:
for i in range(24):
    for j in range(24):
        if abs(sol[i,j]-sol[j,i])>0.01:
            print(i,j)
            print(sol[i,j], sol[j,i])
            print('wrong symmetry')
            
        if abs(sol[i,j]-min(sol[i,i], sol[j,j]))>0.01:
            print(i,j)
            print(sol[i,i], sol[j,j], sol[i,j])
            print('wrong computation')

11 17
0.8893433395380824 0.5875557934505837 0.529969899350303
wrong computation
11 22
0.8893433395380824 0.5230994268276815 0.46573287174205136
wrong computation
17 11
0.5875557934505837 0.8893433395380824 0.529969899350303
wrong computation
17 22
0.5875557934505837 0.5230994268276815 0.30937623427267763
wrong computation
22 11
0.5230994268276815 0.8893433395380824 0.46573287174205136
wrong computation
22 17
0.5230994268276815 0.5875557934505837 0.30937623427267763
wrong computation


## Original (hardcode) method

In [ ]:
print(np.shape(Q_stack))

Q_matrix = []
for i in range(24):
    Q_matrix.append(np.asarray(Q_stack[i]).reshape((4,1)))
# Q: [CA|t1, ..., CA|tN, CB|ti, ..., CC|ti, ...]

FIM_collection = []

for i in range(8):
    # yCA, CA
    mat = sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T
    FIM_collection.append(mat)

for i in range(8):
    # yCA, CB
    FIM_collection.append(sigma_inv[i,i+8]*Q_matrix[i]@Q_matrix[i+8].T)

for i in range(8):
    # YCA, CC
    FIM_collection.append(sigma_inv[i,i+16]*Q_matrix[i]@Q_matrix[i+16].T)

for i in range(8,16): 
    FIM_collection.append(sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T)

for i in range(8,16): 
    FIM_collection.append(sigma_inv[i,i-8]*Q_matrix[i]@Q_matrix[i-8].T)

for i in range(8,16): 
    FIM_collection.append(sigma_inv[i,i+8]*Q_matrix[i]@Q_matrix[i+8].T)

for i in range(16, 24): 
    FIM_collection.append(sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T)

for i in range(16, 24): 
    FIM_collection.append(sigma_inv[i,i-16]*Q_matrix[i]@Q_matrix[i-16].T)

for i in range(16, 24): 
    FIM_collection.append(sigma_inv[i,i-8]*Q_matrix[i]@Q_matrix[i-8].T)
        

print(np.shape(FIM_collection))

In [ ]:
def eval_fim(y, cov_y):
     
    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(y[0])) \
    + sum(yb1*FIM_collection[idx+24] for idx, yb1 in enumerate(y[1])) \
    + sum(yc1*FIM_collection[idx+48] for idx, yc1 in enumerate(y[2])) \
    + sum(yab1*FIM_collection[idx+8] for idx, yab1 in enumerate(cov_y[0])) \
    + sum(yab1*FIM_collection[idx+32] for idx, yab1 in enumerate(cov_y[0])) \
    + sum(yac1*FIM_collection[idx+16] for idx, yac1 in enumerate(cov_y[1])) \
    + sum(yac1*FIM_collection[idx+56] for idx, yac1 in enumerate(cov_y[1])) \
    + sum(ybc1*FIM_collection[idx+40] for idx, ybc1 in enumerate(cov_y[2])) \
    + sum(ybc1*FIM_collection[idx+64] for idx, ybc1 in enumerate(cov_y[2]))
    
    return fim 

def a_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.trace(fim)
    
def d_opt(y, cov_y):
    fim = eval_fim(y, cov_y)
    return cp.log_det(fim)

def e_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return -cp.lambda_min(fim)

In [ ]:
no_res_unit = 8 
cost = [10, 6, 6]

y = []

for idx in range(3):
    create_y = cp.Variable(no_res_unit, nonneg=True)
    y.append(create_y)
    
cov_y = []
for idx in range(3):
    create_cov = cp.Variable(no_res_unit, nonneg=True)
    cov_y.append(create_cov)
    
p_cons = [sum(cp.sum(y[i]*cost[i]) for i in range(3)) <= 100]
#p_cons = [cp.sum(y[0]*10) + cp.sum(y[1]*6) + cp.sum(y[2]*6) <= 100]

#for i in range(3):
for i in y[0]:
    p_cons += [i<=1]
    
for j in y[1]:
    p_cons += [j<=1]
    
for d in y[2]:
    p_cons += [d<=1]
        
p_cons += [cov_y[0][t] <= y[0][t] for t, idx in enumerate(y[0])]
p_cons += [cov_y[0][t] <= y[1][t] for t, idx in enumerate(y[1])]
p_cons += [cov_y[1][t] <= y[0][t] for t, idx in enumerate(y[0])]
p_cons += [cov_y[1][t] <= y[2][t] for t, idx in enumerate(y[2])]
p_cons += [cov_y[2][t] <= y[1][t] for t, idx in enumerate(y[1])]
p_cons += [cov_y[2][t] <= y[2][t] for t, idx in enumerate(y[2])]

p_cons += [y[0][i]+y[1][i]-1<=cov_y[0][i] for i, idx in enumerate(cov_y[0])]
p_cons += [y[0][i]+y[2][i]-1<=cov_y[1][i] for i, idx in enumerate(cov_y[0])]
p_cons += [y[1][i]+y[2][i]-1<=cov_y[2][i] for i, idx in enumerate(cov_y[1])]

obj = cp.Maximize(d_opt(y, cov_y))
#obj = cp.Maximize(d_opt(y[0], y[1], y[2], cov_y[0], cov_y[1], cov_y[2]))

problem = cp.Problem(obj, p_cons)

In [ ]:
print('First time: Optimal value:', problem.solve(verbose=True))

In [ ]:
print(y[0].value)
print(y[1].value)
print(y[2].value)
print(cov_y[0].value)
print(cov_y[1].value)
print(cov_y[2].value)

## Result reader

In [ ]:
time_list = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

def result(li1, li2, li3, verbose=True):
    
  
    rep1 = []
    
    for i, res in enumerate(li1):
        if res >= 0.99:
            #print(str(time_list[i]) + ': yes')
            rep1.append(1)            
        elif res <= 0.01:
            #print(str(time_list[i]) + ': no')
            rep1.append(0)
        else: 
            #print(str(time_list[i]) + ': '+str(res))
            rep1.append(res)

    
    rep2 = []
    
    for i, res in enumerate(li2):
        if res >= 0.99:
            #print(str(time_list[i]) + ': yes')
            rep2.append(1)        
        elif res <= 0.01:
            #print(str(time_list[i]) + ': no')
            rep2.append(0)
        else: 
            #print(str(time_list[i]) + ': '+str(res))
            rep2.append(res)
            
    
    rep3 = []
    
    for i, res in enumerate(li3):
        if res >= 0.99:
            #print(str(time_list[i]) + ': yes')
            rep3.append(1)
        elif res <= 0.01:
            #print(str(time_list[i]) + ': no')
            rep3.append(0)
        else: 
            #print(str(time_list[i]) + ': '+str(res))
            rep3.append(res)
            
            
    if verbose:
        print('CA:', rep1)
        print('CB:', rep2)
        print('CC:', rep3)
            
    return rep1, rep2, rep3

yaa, ybb, ycc = result(y[0].value, y[1].value, y[2].value)
yab1, yac1, ybc1 = result(cov_y[0].value, cov_y[1].value, cov_y[2].value, verbose=False)

def check(ya, yb, yc, yab, yac, ybc):
    
    for i in range(8):
        
        if yab[i] != min(ya[i], yb[i]):
            print('constraint violated: yab', i)
            
        if yac[i] != min(ya[i], yc[i]):
            print('constraint violated: yac', i)
            
        if ybc[i] != min(yc[i], yb[i]):
            print('constraint violated: ybc', i)


    
def estimate_cost(li1, li2, li3):
    print('cost:', sum(li1)*10 + sum(li2)*6 + sum(li3)*6)
    
estimate_cost(yaa, ybb, ycc)
#check(yaa, ybb, ycc, yab1, yac1, ybc1)
print(yab1)
print(yac1)
print(ybc1)

In [ ]:
fim = compute(yaa, ybb, ycc, yab1, yac1, ybc1)
    
print(fim)
print('trace:', np.trace(fim))
print('det:', np.linalg.det(fim))
print('logdet:', np.log(np.linalg.det(fim)))
print('eig:', np.linalg.eigvals(fim))

## round 

In [ ]:
def compute(ya, yb, yc, yab, yac, ybc):

    #print(FIM_collection[33])

    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(ya)) \
        + sum(yb1*FIM_collection[idx+24] for idx, yb1 in enumerate(yb)) \
        + sum(yc1*FIM_collection[idx+48] for idx, yc1 in enumerate(yc)) \
        + sum(yab1*FIM_collection[idx+8] for idx, yab1 in enumerate(yab)) \
        + sum(yab1*FIM_collection[idx+32] for idx, yab1 in enumerate(yab)) \
        + sum(yac1*FIM_collection[idx+16] for idx, yac1 in enumerate(yac)) \
        + sum(yac1*FIM_collection[idx+56] for idx, yac1 in enumerate(yac)) \
        + sum(ybc1*FIM_collection[idx+40] for idx, ybc1 in enumerate(ybc)) \
        + sum(ybc1*FIM_collection[idx+64] for idx, ybc1 in enumerate(ybc))

    return fim



In [ ]:
def generate_cov(la1, lb2, lc3):
    
    lab = []
    
    lac = []
    
    lbc = []
    
    for i in range(8):
        lab.append(min(la1[i], lb2[i]))
        lac.append(min(la1[i], lc3[i]))
        lbc.append(min(lb2[i], lc3[i]))
        
    return lab, lac, lbc

In [ ]:
ya0 = [1, 1, 1, 1, 1, 1, 1, 1]
yb0 = [1, 1, 1, 1, 1, 1, 1, 1]
yc0 = [1, 1, 1, 1, 1, 1, 1, 1]

estimate_cost(ya0, yb0, yc0)
print(sum([sum(ya0), sum(yb0), sum(yc0)]))

yab0, yac0, ybc0 = generate_cov(ya0, yb0, yc0)
print('yab:', yab0)
print('yac:', yac0)
print('ybc:', ybc0)

In [ ]:
fim = compute(ya0, yb0, yc0, yab0, yac0, ybc0)
    
print(fim)
print('trace:', np.trace(fim))
print('det:', np.linalg.det(fim))
print('logdet:', np.log(np.linalg.det(fim)))
print('eig:', np.linalg.eigvals(fim))

## figure 

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
budget = np.linspace(50, 180, 14)
print(budget)

no_m = [7,8,9,11,12,14,15,17,18,20,21,22,23,24]

ln_det = [-0.46, 0.1255, 0.452, 0.9326, 1.2987,
         1.6275, 1.7816, 2.0136, 2.2191, 2.3738, 
         2.4847, 2.5439, 2.6023, 2.6324] 


In [ ]:
fig, ax = plt.subplots()

line1, =ax.plot(budget, ln_det, marker='o', color="red", label='D-optimality')
ax.set_xlabel('Budget ($)', fontsize=18)
ax.set_ylabel('log$_e$(D-optimality)', fontsize=18)
plt.xticks(fontsize=16)
ax.tick_params(axis='y', labelsize=14)

ax2 = ax.twinx()
line2, =ax2.plot(budget, no_m, marker='^', color="blue", label='No. of measurements')
ax2.set_ylabel('No. of measurements', fontsize=18)
ax2.tick_params(axis='y', labelsize=14)

#ax.legend(handles=[line1, line2])
plt.show()

fig.savefig('twinx.jpg',
            format='jpeg',
            dpi=200,
            bbox_inches='tight')

### tests

In [ ]:

test = [[1,0,0,5,0,0,9,0,0],
        [0,1,0,0,5,0,0,9,0],
        [0,0,1,0,0,5,0,0,9],
        [5,0,0,4,0,0,6,0,0],
        [0,5,0,0,4,0,0,6,0],
        [0,0,5,0,0,4,0,0,6],
        [9,0,0,6,0,0,4,0,0],
        [0,9,0,0,6,0,0,4,0],
        [0,0,9,0,0,6,0,0,4]]

test1 = np.asarray(test)

In [ ]:
print(np.linalg.inv(test1))